In [8]:
# imports
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import torch.optim.lr_scheduler as sch

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [18]:
# training
def train_epoch(model,epochs,criterion,optimizer,scheduler,train_loader,val_loader,optimizing=False):
    t_l,v_l=[],[]
    for epoch in range(epochs):

        train_loss=0
        model.train()
        for batch in train_loader:
            X=batch['x'].to(device=Config.device)
            Y=batch['y'].to(device=Config.device)
            out=model(X)
            loss=criterion(out,Y)
            loss.backward()
            optimizer.zero_grad()
            train_loss+=loss.item()
            optimizer.step()
            
        scheduler.step()
        t_i=train_loss/len(train_loader)
        t_l.append(t_i)
      
        if optimizing:
            wandb.log({"loss": loss.item()})
            
        with torch.no_grad():
            val_loss=0
            model.eval()

            for batch in val_loader:
                x=batch['x'].to(device=Config.device)
                y=batch['y'].to(device=Config.device)
                out=model(x)
                loss=criterion(out,y)
                val_loss+=loss.item()
            
            v_i=val_loss/len(val_loader)
            v_l.append(v_i)
            if optimizing:
                wandb.log({'val_batch_loss':v_i})
            
       
    
    return t_l,v_l

- [ ] init weights from a different distribution-(*this potentially changes the loss topography totally removing the local minima you were stuck before*)
- [x] learnignrate scheduler-(*need an initial big learnig rate and then small to get out of local shitty min*)

Create sweep with ID: p5xb1jmm
Sweep URL: https://wandb.ai/theothertom/facial-key/sweeps/p5xb1jmm


wandb: Waiting for W&B process to finish... (success).
wandb: 🚀 View run vibrant-sweep-1 at: https://wandb.ai/theothertom/facial-key/runs/8rqy2k0u
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230403_185625-8rqy2k0u/logs
wandb: Agent Starting Run: ruy53x8j with config:
wandb: 	criterion: l2
wandb: 	epochs: 3
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0.4561636438439619
wandb: 	optim: sgd
wandb: 	scheduler: one_cycle
wandb: 	train_batch_size: 16
wandb: 	val_batch_size: 8


In [15]:
# le scheduler -- give it bruh

In [16]:
# train with best params
from tqdm.autonotebook import trange
def train_with_best_params():
    
    class best_config:
        train_batch_size=16
        val_batch_size=8
        optim='adamw'
        scheduler='rop'
        epochs=50
        criterion='l2'
        learning_rate=1
        momentum=0.4479
        

    model=model_init()
    train_loader,val_loader=data_init(best_config.train_batch_size,best_config.val_batch_size)
    criterion=criterion_init(best_config.criterion)
    optimizer=optimizer_init(model,best_config.optim,best_config.learning_rate,momentum=best_config.momentum)
    scheduler=scheduler_init(best_config.scheduler,optimizer,train_loader,best_config.epochs)

    train_loss,valid_loss=train_epoch(model,epochs,criterion,optimizer,scheduler,train_loader,val_loader)
    t_l.append(train_loss)
    v_l.append(valid_loss)
    return t_l,v_l


t_l,v_l=train_with_best_params()

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 1133, in init
    run = wi.init()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 787, in init
    run_start_result = run_start_handle.wait(timeout=30)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 273, in wait
    found, abandoned = self._slot._get_and_clear(timeout=wait_timeout)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 120, in _get_and_clear
    if self._wait(timeout=timeout):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 116, in _wait
    return self._event.wait(timeout=timeout)
  File "/opt/conda/lib/python3.7/threading.py", line 552, in wait
    signaled = self._cond.wait(timeout)
  File "/opt/conda/lib/python3.7/threading.py", line 300, in wait
    gotit = waiter.acquire(True, timeout)
Exception

During handling of the above exception, a

NameError: name 'epochs' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(t_l)),t_l,'-r');
plt.plot(range(len(v_l)),v_l,'-b');
plt.legend(['avg_train_loss','avg_val_loss']);

In [ ]:
# def plot_preds(model,image_path):
#     from PIL import ImageDraw
#     import torchvision.transforms.functional as tf
    
#     im=Image.open(image_path)
#     draw=ImageDraw.Draw(im)
#     tensor=tf.to_tensor(im)
#     out=model(tensor.to(device=Config.device))
#     out = np.round(out.detach().to('cpu').numpy()).astype(int) #changing real number predictions into nearest integers


#     for feature in out[0]:
#         x,y=feature[0],feature[1]
#         draw.ellipse((x-2,y-2,x+2,y+2),fill=(255,255,255))
#     return im